# Survival Analysis
**What is the motivation for conducting this analysis?**
- To ascertain what percentage of users had churned, and how quickly
- To ascertain the median survival period
- To provide input to retention strategy

**Key findings:**
- data collected between 2018-06-1 ~ 2025-06-01
- 39% of 657,191 users were never activated
- only 5% of activated users had remained active by 2025-06-01
- 41% of activated users had unfollowed/blocked messages from official account
- median survival time: 1,710 days (4.68 years)
- first two weeks are critical to retention 
    - probability of churned >14 days is 79.8% (i.e. ~20% of users had churned by day 14)
    - probability of churned >30 days is 76.8%
- percents of days active, number of entries logged and how many times users logged transactions each week had **no** measurable impact to overall survival duration

**to-do:**
- covariants for `surviva_time`

## Conditions and Assumptions

Users who never logged any transactions are classified as **not-activated** and not included in calculation.

study period: ['2018-06-01', '2025-02-01')

`survival_time = t1 - t0`

`churned`: `is_agree` is `False` 

churn event: user blocked/unfollowed. Users who had not blocked/unfollowed and had not logged any transactions after 365 days are by defaulted **churned**.

right censored: same as `churned` is `False`

if `churned` and `user_ts > last_entry`, make end time = `user_ts`

if not `churned`, make end time = `tsl[1]` end of observation period

? what do it mean when `user_ts` < `last_entry` but `churned` is `True`

## Data Source


In [ ]:
import pandas as pd

users = pd.read_feather('../data/users.feather')
td = pd.read_feather('../data/tidy.feather')

# study period
tsl = pd.to_datetime(['2018-06-01', '2025-06-01'])

# Initial Setup and Analysis

In [ ]:
# y.user_ts.isna() are not-activated users

x = users[(~users.is_bot) & (users.ts < tsl[1])]
y = x.merge(td[['user_id', 'user_ts']], on='user_id', how='left')

print("{size: ,} total users.{not_activated: .0f} percent were never activated."
      .format(not_activated=(y.user_ts.isna()).sum() / x.shape[0] * 100,
              size=x.shape[0])
     )

## Calculate `churned`

In [ ]:
td['days_since'] = tsl[1] - td.last_entry

# set churned flag
td['churned'] = ~td.is_agree
i = td.is_agree & (td.days_since > pd.Timedelta(days=365))
td.loc[i, 'churned'] = True
td.groupby('churned').size()

# churned vs.
# semi-churned, when users blocked but still makes entries past the user_ts flag
td['churned_'] = ~td.is_agree & (td.user_ts < td.last_entry)

In [ ]:
# Was dataset highly censored, which leads bias observations
td.groupby('churned').size()

Was dataset highly censored? **No.**

In [ ]:
# percent churned
print("{:.0f} percent of activated users had churned"
      .format(td.churned.sum() / td.shape[0] * 100))

In [ ]:
# time elapsed since the last transaction entry until the
# observation cut off period tsl[1]
td['days_since'] = tsl[1] - td.last_entry
x = td.loc[td.churned, 'days_since'].dt.total_seconds() / 3600 / 24 # days
_ = x.plot.hist()

In [ ]:
x = td.loc[~td.churned, 'days_since'].dt.total_seconds() / 3600 / 24 # days
_ = x.plot.hist()

In [ ]:
# data issue?
td.query("user_ts.isna()").shape[0]

## Calculate `survival_time`

In [ ]:
# Is left censoring necessary? No.
(td.last_entry < tsl[0]).sum()

In [ ]:
# calculate start time
td['t0'] = td[['user_ts', 'first_entry']].min(axis=1)

calculate end time and survival_time

`tsl[1]` is the observation end time  

if is_agree is False, set the user end time to larger of 
`last_entry` or user record timestamp `ts` from acc_user table

In [ ]:
td['t1'] = tsl[1]
td.loc[~td.is_agree, 't1'] = td.loc[~td.is_agree, ['user_ts', 'last_entry']].max(axis=1, skipna=True)
td['survival_time'] = td.t1 - td.t0

In [ ]:
# how many have churned beyond the observation period
td.loc[td.churned & (td.t1 > tsl[1])].shape[0]

In [ ]:
# how many have churned before the observation period
td[td.churned & (td.t1 < tsl[1])].shape[0]

In [ ]:
# ... and what is the percentage?
td[td.churned & (td.t1 < tsl[1])].shape[0] / td.shape[0]

In [ ]:
# how many have supposedly blocked but still made entries
# vs. true churned(?) user_ts > last_entry
(td[(~td.is_agree) & (td.user_ts < td.last_entry)].shape[0],
td[(~td.is_agree) & (td.user_ts > td.last_entry)].shape[0])

In [ ]:
# is_agree not set and churned flag not set
td.loc[~td.is_agree & td.churned_,
       ['user_ts', 'last_entry', 'tenure', 'days_active',
        'days_since', 'survival_time', 't1', 't0',
        'churned_', 'user_id']]

In [ ]:
# churned vs blocked
print("{churned:.0f} percent churned vs. {blocked:.0f} percent blocked"
    .format(churned=td.churned.sum() / td.shape[0] * 100,
            blocked=(~td.is_agree).sum() / td.shape[0] * 100)
     )

In [ ]:
td[['tenure', 'days_active', 'days_since', 'survival_time', 'nbr_entry']].describe()

In [ ]:
# spread between tenure and active days
(td.tenure.dt.days - td.days_active).describe()

_**Oberservations:**_
- 75% of users had been active for 12 separate days or less (`days_active`)
- 75% of users had not made any entries for >1143 days (`days_since`)

In [ ]:
# what do you observe from the top quartile...

t0 = td.days_since.quantile(.25) # 1146 days since last entry
print('days_since =', t0.days)
(td.loc[td.days_since < t0, ['days_since', 'tenure', 'days_active', 'nbr_entry']].describe())

In [ ]:
td[td.user_id == 'U000046b3786c997220a07872c5191c37']

# Kaplan-Meier Estimator

In [ ]:
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

kmf = KaplanMeierFitter()
kmf.fit(durations=td.survival_time.dt.total_seconds() / 3600 / 24, # in days
        event_observed=td.churned)

ax = kmf.plot_survival_function()
ax.set_title("Kaplan-Meier Survival Curve")
ax.set_xlabel("days")
ax.set_ylabel("Survival Probability")
plt.show()

In [ ]:
median_time = kmf.median_survival_time_
print(f"Median Survival Time: {median_time:.0f} days")

In [ ]:
# Find duration for a given probability
target_prob = 0.768  # survival probability 
closest_time = kmf.survival_function_.index[kmf.survival_function_["KM_estimate"] <= target_prob][0]

print(f"Duration corresponding to survival probability {target_prob}: {closest_time:.1f} days")

In [ ]:
# Find duration for a given probability
target_prob = 0.798 # survival probability 
closest_time = kmf.survival_function_.index[kmf.survival_function_["KM_estimate"] <= target_prob][0]

print(f"Duration corresponding to survival probability {target_prob}: {closest_time:.1f} days")

# Cox Proportional Hazards Model

In [ ]:
from lifelines import CoxPHFitter

td['duration'] = td.survival_time.dt.total_seconds() / 3600 / 24  # days
df = td[[
    'duration',
    'churned',
    'pct_active',
    'nbr_entry',
    'fq_median'
]]

cph = CoxPHFitter()
cph.fit(df, duration_col='duration', event_col='churned')
cph.print_summary()

**Observation:** percents of days active, number of entries logged and how many times users logged transactions each week had no measurable impact to overall survival duration.

In [ ]:
td.info()